Google Colab Setup

In [1]:
# setup
!pip install -q tensorflow-text

     |████████████████████████████████| 4.3MB 7.7MB/s 


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# Load pretrain classifier

In [3]:
#model_name = 'bert_Emotion_Stimulus.h5'
#class_names = ['anger', 'disgust', 'fear', 'happy', 'sad', 'shame', 'surprise']

#model_name = 'bert_daily_dialog.h5'
#class_names = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']

#model_name = 'bert_ISEAR.h5'
#class_names = ['anger', 'fear', 'joy', 'sadness']

model_name = 'bert_affective_text.h5'
class_names = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

In [4]:
my_reloaded_model = tf.keras.models.load_model((model_name),
                                               custom_objects={'KerasLayer':hub.KerasLayer})


In [5]:
def get_emotions(sentences: list)->list:
  emotions = my_reloaded_model.predict(sentences)
  return [ (sentence, class_names[prediction.argmax()], prediction.max()) 
  for sentence, prediction in zip(sentences, emotions)]

# Load & proprocess datsets

In [10]:
from preprocess import EmotionStimulus, DailyDialog, ISEAR, AffectiveText
import shutil
import pandas as pd
shutil.unpack_archive('Datasets.zip')

In [8]:
def get_strength(sentences, emotions):
  emotion_predictions = get_emotions(sentences)
  sentence_strength = list()
  for prediction, emotion in zip(emotion_predictions, emotions):
    if prediction[1] == emotion:
      s_strength = prediction[2]
    else:
      s_strength = 0
    sentence_strength.append([prediction[0], emotion, s_strength])
  return sentence_strength

In [11]:
#preprocessor = EmotionStimulus('Datasets/Emotion_Cause/')
#preprocessor = DailyDialog('Datasets/ijcnlp_dailydialog')
#preprocessor = ISEAR('Datasets/eng_dataset.csv')
preprocessor = AffectiveText('Datasets/AffectiveText.Semeval.2007/')


texts = preprocessor.sentences
emotions = preprocessor.tags



In [12]:
len(texts),len(emotions)

(1246, 1246)

In [13]:
sentence_emotion_strength = get_strength(texts, emotions)

In [14]:
df = pd.DataFrame(sentence_emotion_strength, columns=['text', 'emotion','strength'])
df.to_csv('AffectiveText_strength.csv')

In [15]:
df

,text,emotion,strength
0,Test to predict breast cancer relapse is approved,joy,0.997365
1,"Two Hussein allies are hanged, Iraqi official ...",sadness,0.998487
2,Sights and sounds from CES,joy,0.997521
3,Schuey sees Ferrari unveil new car,joy,0.996815
4,Closings and cancellations top advice on flu o...,fear,0.938222
...,...,...,...
1241,Amish killings school demolished,sadness,0.995132
1242,Pamuk wins Nobel Literature prize,joy,0.996581
1243,Madonna adoption bid approved,joy,0.997431
1244,Man admits UK-US terror bomb plot,fear,0.000000
